In [22]:
# Import the relevent packages
import urllib.request
from datetime import date
from pathlib import Path
import pandas as pd
import numpy as np

import ipywidgets as widgets


In [23]:
# Download the dataset

## Make a new folder for each day's data
todayfolder = date.today().strftime("%d/%m/%Y").replace('/','.')
Path(todayfolder).mkdir(parents=True, exist_ok=True)

# Reference to file names being created
datafiles = {'confirmed':f'{todayfolder}/time_series_2019-ncov-Confirmed.csv',
            'deaths':f'{todayfolder}/time_series_2019-ncov-Deaths.csv',
            'recoverd':f'{todayfolder}/time_series_2019-ncov-Recovered.csv'}

# if the data has not been downloaded, download it

if not Path(datafiles['confirmed']).is_file():
    urllib.request.urlretrieve('https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_19-covid-Confirmed.csv&filename=time_series_2019-ncov-Confirmed.csv', datafiles['confirmed'])

if not Path(datafiles['deaths']).is_file():
    urllib.request.urlretrieve('https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_19-covid-Deaths.csv&filename=time_series_2019-ncov-Deaths.csv', datafiles['deaths'])

if not Path(datafiles['recoverd']).is_file():
    urllib.request.urlretrieve('https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_19-covid-Recovered.csv&filename=time_series_2019-ncov-Recovered.csv', datafiles['recoverd'])

print('Downloaded latest data')


Downloaded latest data


In [60]:
# Read the data files

conf_df = pd.read_csv(datafiles['confirmed'])
death_df = pd.read_csv(datafiles['deaths'])
cure_df = pd.read_csv(datafiles['recoverd'])

# Add identification for each dataset
dateCols = conf_df.columns[conf_df.columns.get_loc("Long")+1:]
conf_df['Attribute'] = 'confirmed'
death_df['Attribute'] = 'deaths'
cure_df['Attribute'] = 'cured'

# Change the order of columns for each dataset
newCols = ['Country/Region','Province/State','Attribute','Lat','Long']
conf_df = conf_df[[*newCols, *dateCols]]
death_df = death_df[[*newCols, *dateCols]]
cure_df = cure_df[[*newCols, *dateCols]]

In [59]:
#df = df[df['Country/Region'] == selectedCountry.value]
aggregate_window = 7

# Get the number of active cases
active_df = conf_df.copy()
active_df[dateCols] = active_df[dateCols].values - death_df[dateCols].values - cure_df[dateCols].values

# Calculate the n day moving average
avr_window = aggregate_window
conf_avr_df = conf_df.copy()
conf_avr_df[dateCols] = conf_avr_df[dateCols].iloc[:,:].rolling(window=avr_window, axis = 1).mean()

# Calculate the number of new cases per day
conf_new_df = conf_df.copy()
conf_new_df[dateCols] = conf_new_df[dateCols].iloc[:,:].rolling(window=2, axis = 1).apply(lambda x: x[-1] - x[0],raw=True)

# Calculate the n day diffrence
diff_Window = aggregate_window
conf_nday_diff_df = conf_df.copy()
conf_nday_diff_df[dateCols] = conf_nday_diff_df[dateCols].iloc[:,:].rolling(window=aggregate_window, axis = 1).apply(lambda x: x[-1] - x[0],raw=True)

# Calculate the growth rate per day
conf_new_pct_df = conf_df.copy()
conf_new_pct_df[dateCols] = conf_new_pct_df[dateCols].iloc[:,:].rolling(window=2, axis = 1).apply(lambda x: (x[1] - x[0])/x[0],raw=True)


# Calculate the n day average growth rate
avr_nday_growth_window = aggregate_window
conf_nday_avr_new_df = conf_new_pct_df.copy()
conf_nday_avr_new_df[dateCols] = conf_nday_avr_new_df[dateCols].iloc[:,:].rolling(window=aggregate_window, axis = 1).mean()


In [55]:
countryList = conf_df['Country/Region'].unique()
selectedCountry = widgets.Dropdown(
    options=countryList,
    value='Australia',
    description='Country:',
    disabled=False,
)
selectedCountry

,Country/Region,Province/State,Attribute,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20
0,Thailand,NaN,confirmed,15.0000,101.0000,NaN,NaN,NaN,NaN,NaN,...,0.060370,0.067535,0.074916,0.130665,0.172018,0.192601,0.204677,0.218474,0.234531,0.260683
1,Japan,NaN,confirmed,36.0000,138.0000,NaN,NaN,NaN,NaN,NaN,...,0.086852,0.076904,0.077631,0.077123,0.072178,0.061786,0.049315,0.054939,0.047108,0.038962
2,Singapore,NaN,confirmed,1.2833,103.8333,NaN,NaN,NaN,NaN,NaN,...,0.062682,0.064465,0.064246,0.061257,0.072003,0.076001,0.085171,0.099776,0.098683,0.107551
3,Nepal,NaN,confirmed,28.1667,84.2500,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Malaysia,NaN,confirmed,2.5000,112.5000,NaN,NaN,NaN,NaN,NaN,...,0.183488,0.135224,0.147744,0.252573,0.272660,0.285015,0.287702,0.307593,0.282207,0.273696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,Canada,Northwest Territories,confirmed,64.8255,-124.8457,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
478,Cape Verde,NaN,confirmed,15.1111,-23.6167,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
479,East Timor,NaN,confirmed,-8.5500,125.5600,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480,Eritrea,NaN,confirmed,15.1794,39.7823,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
conf_nday_avr_new_df[conf_new_pct_df['Country/Region'] == selectedCountry.value]

,Country/Region,Province/State,Attribute,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20
6,Australia,New South Wales,confirmed,-33.8688,151.2093,NaN,NaN,NaN,NaN,NaN,...,0.172385,0.205751,0.225818,0.202859,0.204711,0.216459,0.229260,0.250662,0.212727,0.215261
7,Australia,Victoria,confirmed,-37.8136,144.9631,NaN,NaN,NaN,NaN,NaN,...,0.118615,0.220656,0.257957,0.281281,0.264420,0.282127,0.299351,0.299351,0.197310,0.273231
8,Australia,Queensland,confirmed,-28.0167,153.4000,NaN,NaN,NaN,NaN,NaN,...,0.066422,0.173565,0.218463,0.243069,0.259463,0.251900,0.265331,0.341318,0.273858,0.257687
19,Australia,South Australia,confirmed,-34.9285,138.6007,NaN,NaN,NaN,NaN,NaN,...,0.097959,0.151927,0.178713,0.186232,0.250518,0.250518,0.249110,0.268415,0.184515,0.206301
22,Australia,From Diamond Princess,confirmed,35.4437,139.6380,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,Australia,Western Australia,confirmed,-31.9505,115.8605,NaN,NaN,NaN,NaN,NaN,...,0.190476,0.269841,0.300454,0.300454,0.345271,0.289149,0.236154,0.305541,0.259143,0.286567
61,Australia,Tasmania,confirmed,-41.4545,145.9707,NaN,NaN,NaN,NaN,NaN,...,0.214286,0.309524,0.309524,0.195238,0.219048,0.219048,0.208844,0.208844,0.113605,0.199320
71,Australia,Northern Territory,confirmed,-12.4634,130.8456,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.285714,0.285714
402,Australia,Australian Capital Territory,confirmed,-35.4735,149.0124,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,0.404762
